## Import packages

In [1]:
# Single dataset
import arviz as az  # Visualization
import bambi as bmb  #
import hssm
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt
import pymc as pm
import requests
from io import StringIO
from joblib import Parallel, delayed
from hssm.distribution_utils import make_distribution
from hssm.likelihoods import DDM

In [2]:
hssm.set_floatX("float64")
#hssm.set_floatX("float32")

Setting PyTensor floatX type to float64.
Setting "jax_enable_x64" to True. If this is not intended, please set `jax` to False.


## Set variables

In [3]:
num_draws = 4000
num_tune = 4000 
targ_accept = 0.99

## Helper functions

In [4]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [5]:
def FormatDataDriftRateModels(df):    
    df_par = df[(df['ParticipantID'] == par)]
    df_par_sorted = df_par.sort_values(by='ExperimentRoundNumber')
    n_trials = len(df_par)
    
    par_data = {}

    par_rts = []
    par_responses = []
    diff_x_list = []
    diff_p_list = []
    diff_ev_list = []

    for i in range(n_trials):
    
        df_par_round = df_par.iloc[i]

        #Retrieve reaction time and choices
        par_rts.append(df_par_round['ResponseTimeGamble'])

        choice = df_par_round['GambleChoice']
    
        if choice == 'Left':
            lot = df_par_round['LotteryLeft']
    
        if choice == 'Right':
            lot = df_par_round['LotteryRight']
    
        if lot == 0:
            par_responses.append(-1)
        if lot == 1:
            par_responses.append(1)

        gamble_num = df_par_round['ExperimentGambleNumber']

        gamble_df = df_gambles[df_gambles['GambleNumber']==gamble_num]

        #diff_x = gamble_df["lot_0_val"].iloc[0] - gamble_df["lot_1_val"].iloc[0]
        #diff_p = gamble_df["lot_0_prob"].iloc[0] - gamble_df["lot_1_prob"].iloc[0]
        diff_x = gamble_df["lot_1_val"].iloc[0] - gamble_df["lot_0_val"].iloc[0]
        diff_p = gamble_df["lot_1_prob"].iloc[0] - gamble_df["lot_0_prob"].iloc[0]

        diff_x = diff_x/100
        diff_p = diff_p/100

        ev_0 = (gamble_df["lot_0_val"].iloc[0]/100) * (gamble_df["lot_0_prob"].iloc[0]/100)
        ev_1 = (gamble_df["lot_1_val"].iloc[0]/100) * (gamble_df["lot_1_prob"].iloc[0]/100)

        #diff_ev = ev_0 - ev_1
        diff_ev = ev_1 - ev_0

        diff_x_list.append(diff_x)
        diff_p_list.append(diff_p)
        diff_ev_list.append(diff_ev)
    
    par_data["diff_x"] = diff_x_list 
    par_data["diff_p"] = diff_p_list 
    par_data["diff_ev"] = diff_ev_list  
    par_data["response"] = par_responses
    par_data["rt"] = par_rts

    df_par_data = pd.DataFrame(par_data)

    return df_par_data

## Import data and gambles

In [6]:
try:
    df_processed_risky_dm_task_data = pd.read_csv(
        "../data_preprocessing/processed_data/processed_risky_dm_task_data.csv"
    )
    print("Successfully loaded processed_risky_dm_task_data.csv")
except FileNotFoundError:
    print("File not found. Please run the preprocessing pipeline first to generate 'processed_risky_dm_task_data.csv'.")
except Exception as e:
    print(f"An error occurred while loading the data: {e}")

Successfully loaded processed_risky_dm_task_data.csv


In [7]:
username = 'algarrid'
password = 'faunistico'

# URL of the CSV file 
file_url = 'https://cloud.ilabt.imec.be/remote.php/dav/files/af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/cwir_dataset/stimuli/gambles.csv'

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_gambles = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Gambles downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

Gambles downloaded


In [8]:
par_ids = np.unique(df_processed_risky_dm_task_data['ParticipantID'])
n_par = len(par_ids)

## Define fitting function

In [9]:
def fit_ddm(par):

    model_name = f"model_{par}"

    df_par = df_processed_risky_dm_task_data[df_processed_risky_dm_task_data['ParticipantID'] == par]
    
    df_formatted_data = FormatDataDriftRateModels(df_par)

    fitted_params = {}

    with pm.Model() as model_name:
        a = pm.Uniform("a", lower=0.5, upper=3.5)
        t = pm.HalfNormal("t", sigma = 0.3)
        v_diff_x = pm.HalfNormal("v_diff_x", sigma=3)
        v_diff_p = pm.HalfNormal("v_diff_p", sigma=3)
        v_diff_ev = pm.HalfNormal("v_diff_ev", sigma=3)

        diff_x = df_formatted_data['diff_x'].values
        diff_p = df_formatted_data['diff_p'].values
        diff_ev = df_formatted_data['diff_ev'].values


        v = v_diff_x*diff_x + v_diff_p*diff_p + v_diff_ev*diff_ev

        ddm = DDM(
            "DDM", v=v, a=a, z=0.5, t=t, observed=df_formatted_data[["rt", "response"]].values
        )
    
        ddm_trace = pm.sample(draws = num_draws, 
                                     tune=num_tune, 
                                     target_accept = targ_accept, 
                                     return_inferencedata=True)

    fitted_params['ParticipantID'] = par

    fitted_params['fitted_a_mean'] = az.summary(ddm_trace)['mean']['a']
    fitted_params['fitted_a_std'] = az.summary(ddm_trace)['sd']['a']
    fitted_params['fitted_a_r_hat'] = az.summary(ddm_trace)['r_hat']['a']
    
    fitted_params['fitted_t_mean'] = az.summary(ddm_trace)['mean']['t']
    fitted_params['fitted_t_std']= az.summary(ddm_trace)['sd']['t']
    fitted_params['fitted_t_r_hat'] = az.summary(ddm_trace)['r_hat']['t']
        
    fitted_params['fitted_v_diff_p_mean'] = az.summary(ddm_trace)['mean']['v_diff_p']
    fitted_params['fitted_v_diff_p_std'] = az.summary(ddm_trace)['sd']['v_diff_p']
    fitted_params['fitted_v_diff_p_r_hat'] = az.summary(ddm_trace)['r_hat']['v_diff_p']
        
    fitted_params['fitted_v_diff_x_mean'] = az.summary(ddm_trace)['mean']['v_diff_x']
    fitted_params['fitted_v_diff_x_std'] = az.summary(ddm_trace)['sd']['v_diff_x']
    fitted_params['fitted_v_diff_x_r_hat'] = az.summary(ddm_trace)['r_hat']['v_diff_x']

    fitted_params['fitted_v_diff_ev_mean'] = az.summary(ddm_trace)['mean']['v_diff_ev']
    fitted_params['fitted_v_diff_ev_std'] = az.summary(ddm_trace)['sd']['v_diff_ev']
    fitted_params['fitted_v_diff_ev_r_hat'] = az.summary(ddm_trace)['r_hat']['v_diff_ev']

    return fitted_params, ddm_trace

## Fit parameters

In [10]:
fitted_params = []
traces = []

for par in tqdm(par_ids):
    
    fitting_result, trace = fit_ddm(par)

    fitted_params.append(fitting_result)

    traces.append(trace)
 

  0%|          | 0/21 [00:00<?, ?it/s]<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> Cannot convert Type Matrix(float32, shape=(2, ?)) (of Variable Switch.0) into Type Matrix(float64, shape=(2, ?)). You can try to manually convert Switch.0 into a Matrix(float64, shape=(2, ?)). local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): node: Mul(Eq.0, Switch.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1966, in process_node
    fgraph.replace_all_validate_remove(  # type: ignore
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/features.py", line 628, in replace_all_validate_remove
    chk = fgraph.replace_all_validate(replacements, reason=reason, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Output()

Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 187 seconds.
  5%|▍         | 1/21 [05:55<1:58:22, 355.11s/it]<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> Cannot convert Type Matrix(float32, shape=(2, ?)) (of Variable Switch.0) into Type Matrix(float64, shape=(2, ?)). You can try to manually convert Switch.0 into a Matrix(float64, shape=(2, ?)). local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): node: Mul(Eq.0, Switch.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1966, in process_node
    fgraph.replace_all_validate_remove(  # type: ignore
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/features.py", line 628, in replace_all_validate_remove


Output()

Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 180 seconds.
 10%|▉         | 2/21 [09:10<1:22:45, 261.34s/it]<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> Cannot convert Type Matrix(float32, shape=(2, ?)) (of Variable Switch.0) into Type Matrix(float64, shape=(2, ?)). You can try to manually convert Switch.0 into a Matrix(float64, shape=(2, ?)). local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): node: Mul(Eq.0, Switch.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1966, in process_node
    fgraph.replace_all_validate_remove(  # type: ignore
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/features.py", line 628, in replace_all_validate_remove


...

Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 156 seconds.
 90%|█████████ | 19/21 [1:05:24<06:28, 194.47s/it]<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> Cannot convert Type Matrix(float32, shape=(2, ?)) (of Variable Switch.0) into Type Matrix(float64, shape=(2, ?)). You can try to manually convert Switch.0 into a Matrix(float64, shape=(2, ?)). local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): node: Mul(Eq.0, Switch.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1966, in process_node
    fgraph.replace_all_validate_remove(  # type: ignore
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/features.py", line 628, in replace_all_validate_remove

Output()

Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 144 seconds.
 95%|█████████▌| 20/21 [1:08:05<03:04, 184.32s/it]<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> Cannot convert Type Matrix(float32, shape=(2, ?)) (of Variable Switch.0) into Type Matrix(float64, shape=(2, ?)). You can try to manually convert Switch.0 into a Matrix(float64, shape=(2, ?)). local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_mul_switch_sink
ERROR (pytensor.graph.rewriting.basic): node: Mul(Eq.0, Switch.0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1966, in process_node
    fgraph.replace_all_validate_remove(  # type: ignore
  File "/opt/conda/lib/python3.11/site-packages/pytensor/graph/features.py", line 628, in replace_all_validate_remove

Output()

Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 120 seconds.
100%|██████████| 21/21 [1:10:21<00:00, 201.01s/it]


## Save fitted parameters

In [12]:
df_parameter_fitting_results = pd.DataFrame(fitted_params)
df_parameter_fitting_results.to_csv("fitted_parameters/ddm/fitted_parameters_ddm.csv", index=False)